In [1]:
import torch
from flash_attn import flash_attn_varlen_func

# small test
q = torch.randn(6, 8, 64, device='cuda', dtype=torch.float16).contiguous()  # sum_seq=6, nheads=8, head_dim=64
k = torch.randn(6, 4, 64, device='cuda', dtype=torch.float16).contiguous()
v = torch.randn(6, 4, 64, device='cuda', dtype=torch.float16).contiguous()
cu_seqlens = torch.tensor([0, 3, 6], dtype=torch.int32, device='cuda')  # two sequences: lengths 3 and 3
out = flash_attn_varlen_func(q, k, v, cu_seqlens_q=cu_seqlens, cu_seqlens_k=cu_seqlens, max_seqlen_q=3, max_seqlen_k=3, dropout_p=0.0, causal=True)
print("flash test ok, out.shape=", out.shape)

flash test ok, out.shape= torch.Size([6, 8, 64])
